CVPRO v1.0.0.<br>
Code Developed by Augustin Rajkumar, Suresh Balaji, E.V.V Thrilok kumar, and Meritus R & D Team - August 31, 2023.<br>
Copyright © 2023 Meritus R & D Team. All rights reserved.<br>
This program is the intellectual property of Meritus AI, and may not be distributed 
or reproduced without explicit authorization from the copyright holder.<br>
This notebook helps in training your model based on your data collected

Import packages

In [ ]:
from Utils.createdataset import CreateDataset
from Utils.encoder import Labeling
from Utils.train import TrainingModel

Parameters of Batch-size, Epochs and Learning-rate

In [ ]:
batch_size = 32
epochs = 5
learning_rate = 0.0001

Create a Dataset

In [ ]:
preprocess = CreateDataset()
root_folder, log_file_path = preprocess.main_folder()
folder_list = preprocess.path_folders()

Image_PreProcessing

In [ ]:
preprocess.video_to_image(folder_list)

To save the dataframe of each folder

In [ ]:
save_df = preprocess.data_frame()
csv_path = preprocess.list_to_dataframe(save_df)

To show the random_images from the datasets

In [ ]:
preprocess.random_images(csv_path)

Encoding the labels

In [ ]:
encode_label = Labeling(root_folder, csv_path)
data, encoding_label, unique_values, decoding_label = encode_label.encoder_label()

To show the plot of labels with their counts

In [ ]:
encode_label.show_encode_plot()

No.of Classes for the training neural network

In [ ]:
num_classes = encode_label.num_label()

Getting Images and Controls values

In [ ]:
images_path, controls = encode_label.load_data()

Create an instance of TrainingModel

In [ ]:
training_model = TrainingModel(root_folder, images_path, controls, num_classes)

Split the data

In [ ]:
x_train, x_val, x_test, y_train, y_val, y_test = training_model.split_data()

Create the custom data generator

In [ ]:
train_generator = training_model.CustomDataGenerator(x_train, y_train, batch_size)
val_generator = training_model.CustomDataGenerator(x_val, y_val, batch_size)
test_generator = training_model.CustomDataGenerator(x_test, y_test, batch_size)

Create the model

In [ ]:
model = training_model.create_model(learning_rate)

Train the model

In [ ]:
history, best_checkpoint, last_checkpoint = training_model.train(model, epochs, train_generator, val_generator)

Evaluate the model

In [ ]:
training_model.evaluate(model, test_generator)

Predict with the model

In [ ]:
true_labels, predict_classes = training_model.predict(model, test_generator)

Calculate metrics

In [ ]:
training_model.conf_matrix(true_labels, predict_classes)
ACCURACY = training_model.train_metrics(true_labels, predict_classes)

Convert to TFLite

In [ ]:
training_model.convert_to_tflite(ACCURACY, best_checkpoint, last_checkpoint)

Save labels

In [ ]:
training_model.save_labels(decoding_label)